In [1]:
import refinitiv.data as rd
import numpy as np
import pandas as pd
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fd6be5e81c0 {name='codebook'}>

In [3]:
# which index is used as a basis of comparison
index = '.SPX'
# what is the time range for the backtesting 
start_year = 2014
end_year = 2023
# what is the company's  minimum ESG Score (0-100) for the high ESG subset
ESG_score_threshold = 70



In [4]:
sp = rd.get_data(index, fields=['TR.PRICECLOSE.date', 'TR.PRICECLOSE'], parameters={'SDate': '{}-01-01'.format(start_year), 'EDate': '{}-01-01'.format(end_year), 'Frq':'CQ'})

In [5]:
sp['Rebased'] = sp['Price Close'] * 100 / sp['Price Close'][0]
sp.head()

,Instrument,Date,Price Close,Rebased
0,.SPX,2014-03-31,1872.34,100.0
1,.SPX,2014-06-30,1960.23,104.694126
2,.SPX,2014-09-30,1972.29,105.33824
3,.SPX,2014-12-31,2058.9,109.964002
4,.SPX,2015-03-31,2067.89,110.44415


In [6]:
sp.to_csv('SnP500.csv', index=False)

## Getting Data from Companies

first get the companies that has esg coverage, then we'll get their price close.

In [7]:
# Europe, Africa, and Oceania
df_eur_Af_oce = rd.get_data(["SCREEN(U(IN(Equity(active,public,primary))/*UNV:Public*/), TR.HasESGCoverage==true, IN(TR.HeadquartersRegion,""Europe"",""Africa"",""Oceania""), CURN=USD)"],["TR.CommonName;TR.HasESGCoverage;TR.HeadquartersRegion"])


# Asia
df_asia = rd.get_data(["SCREEN(U(IN(Equity(active,public,primary))/*UNV:Public*/), TR.HasESGCoverage==true, IN(TR.HeadquartersRegion,""Asia""), CURN=USD)"],["TR.CommonName;TR.HasESGCoverage;TR.HeadquartersRegion"])


# Americas
df_am = rd.get_data(["SCREEN(U(IN(Equity(active,public,primary))/*UNV:Public*/), TR.HasESGCoverage==true,  IN(TR.HeadquartersRegion,""Americas""), CURN=USD)"],["TR.CommonName;TR.HasESGCoverage;TR.HeadquartersRegion"])

# concat all
esg_All = pd.concat([df_eur_Af_oce, df_asia, df_am], ignore_index=True)
esg_All

,Instrument,Company Common Name,ESG Coverage Flag,Region of Headquarters
0,MBWS.PA,Marie Brizard Wine and Spirits SA,True,Europe
1,AFEJ.J,AECI Ltd,True,Africa
2,AELJ.J,Altron Ltd,True,Africa
3,AAL.L,Anglo American PLC,True,Europe
4,AVIJ.J,Avi Ltd,True,Africa
...,...,...,...,...
14534,BTSG.OQ,Brightspring Health Services Inc,True,Americas
14535,IBTA.N,Ibotta Inc,True,Americas
14536,RBRK.N,Rubrik Inc,True,Americas
14537,INBX.OQ,Inhibrx Biosciences Inc,True,Americas


In [11]:
esg_All.to_csv("iNS_esgcov_regionHQ.csv")

In [20]:
def chunks(l, n):
    for i in range(0,len(l),n):
        yield l[i:i+n]
        
def data_to_dict(df3, my_dict, empty_rics):
    rics = []
    [rics.append(ric[0]) for ric in df3.columns]
    rics = list(set(rics))
    for ric in rics:
        df3_ric = df3[ric].dropna(subset = ['ESG Combined Score'])
        if len(df3_ric) > 0:
            my_dict[ric] = {'date_min':'2014-01-01', 'date_max':df3_ric['ESG Period Last Update Date'][-1]}
        else:
            empty_rics.append(ric)
    return my_dict, empty_rics


ric_chunks = list(chunks(list(esg_All['Instrument'].values),1000))
i = 0
my_dict ={}
empty_rics = []
for chunk in ric_chunks:
    i+=1
    print(len(ric_chunks)-i)
    max_retry = 0
    while max_retry<3:
        try:
            df3 = rd.get_history(chunk,['TR.TRESGCScore(SDate=0,EDate=-19,Period=FY0,Frq=FY)',
                                        'TR.ESGPeriodLastUpdateDate(SDate=0,EDate=-19,Period=FY0,Frq=FY)'])
            if len(df3) >0:
                my_dict, empty_rics = data_to_dict(df3, my_dict, empty_rics)
        except Exception as e:
            print(e)
            max_retry+=1
            continue
        
        break
        


14
13


KeyboardInterrupt: 

In [21]:
len(esg_All)

14539

In [22]:
pd.DataFrame(my_dict)

,KSBG.DE,SAN.NZ,HAYS.L,OMNJ.J,RCSM.MI,TUI1n.DE,CARRC.L,NICL.L,HEIN.AS,TSGJ.J,...,MTO.L,ELD.AX,ACCP.PA,PSHG_p.DE,SVEG.OL,AEGN.AS,VOSG.DE,AZK.MC,RBREW.CO,MARS.L
date_min,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,...,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01
date_max,2024-11-29,2024-11-22,2024-11-22,2024-11-22,2024-11-22,2024-11-22,2024-11-22,2024-11-22,2024-11-14,2024-11-22,...,2024-11-22,2024-11-22,2024-11-22,2024-11-29,2024-11-22,2024-08-23,2024-11-14,2024-11-22,2024-11-22,2024-11-22


## Get ESG Scores

In [33]:
esg_Scores = pd.DataFrame()
i = 0
for chunk in ric_chunks:
    i+=1
    print(len(ric_chunks)-i)
    max_retry = 0
    while max_retry<3:
        try:
            df3x = rd.get_history(chunk,['TR.TRBCEconomicSector',
                                         'TR.TRESGScore(Period=FY0)',
                                         'TR.TRESGCScore(Period=FY0)',
                                         'TR.TRESGResourceUseScore(Period=FY0)',
                                         'TR.TRESGEmissionsScore(Period=FY0)',
                                         'TR.TRESGInnovationScore(Period=FY0)',
                                         'TR.TRESGWorkforceScore(Period=FY0)',
                                         'TR.TRESGHumanRightsScore(Period=FY0)',
                                         'TR.TRESGCommunityScore(Period=FY0)',
                                         'TR.TRESGProductResponsibilityScore(Period=FY0)',
                                         'TR.TRESGCControversiesScore(Period=FY0)'])
            if len(df3x):
                esg_Scores = pd.concat([esg_Scores,df3x])
            else:
                esg_Scores = df3x
        except Exception as e:
            print(e)
            continue
        break
        


14
13


KeyboardInterrupt: 

In [36]:
esg_Scores

MBWS.PA                                \
           TRBC Economic Sector Name  ESG Score ESG Combined Score   
Date                                                                 
2008-09-30                      <NA>       <NA>               <NA>   
2019-12-31                      <NA>       <NA>               <NA>   
2020-11-30                      <NA>       <NA>               <NA>   
2020-12-31                      <NA>       <NA>               <NA>   
2021-03-31                      <NA>       <NA>               <NA>   
2021-12-31                      <NA>       <NA>               <NA>   
2022-01-29                      <NA>       <NA>               <NA>   
2022-01-31                      <NA>       <NA>               <NA>   
2022-03-31                      <NA>       <NA>               <NA>   
2022-05-31                      <NA>       <NA>               <NA>   
2022-06-30                      <NA>       <NA>               <NA>   
2022-08-31                      <NA>       <NA>               <NA>   
2022-09-03                      <NA>       <NA>               <NA>   
2022-09-30                      <NA>       <NA>               <NA>   
2022-10-01                      <NA>       <NA>               <NA>   
2022-10-31                      <NA>       <NA>               <NA>   
2022-11-30                      <NA>       <NA>               <NA>   
2022-12-31                      <NA>       <NA>               <NA>   
2023-01-01                      <NA>       <NA>               <NA>   
2023-01-28                      <NA>       <NA>               <NA>   
2023-01-29                      <NA>       <NA>               <NA>   
2023-01-31                      <NA>       <NA>               <NA>   
2023-02-25                      <NA>       <NA>               <NA>   
2023-02-26                      <NA>       <NA>               <NA>   
2023-02-28                      <NA>       <NA>               <NA>   
2023-03-02                      <NA>       <NA>               <NA>   
2023-03-04                      <NA>       <NA>               <NA>   
2023-03-25                      <NA>       <NA>               <NA>   
2023-03-31                      <NA>       <NA>               <NA>   
2023-04-01                      <NA>       <NA>               <NA>   
2023-04-02                      <NA>       <NA>               <NA>   
2023-04-03                      <NA>       <NA>               <NA>   
2023-04-28                      <NA>       <NA>               <NA>   
2023-04-30                      <NA>       <NA>               <NA>   
2023-05-31                      <NA>       <NA>               <NA>   
2023-06-25                      <NA>       <NA>               <NA>   
2023-06-30                      <NA>       <NA>               <NA>   
2023-07-02                      <NA>       <NA>               <NA>   
2023-07-29                      <NA>       <NA>               <NA>   
2023-07-31                      <NA>       <NA>               <NA>   
2023-08-01                      <NA>       <NA>               <NA>   
2023-08-31                      <NA>       <NA>               <NA>   
2023-09-30                      <NA>       <NA>               <NA>   
2023-10-31                      <NA>       <NA>               <NA>   
2023-11-30                      <NA>       <NA>               <NA>   
2023-12-30                      <NA>       <NA>               <NA>   
2023-12-31                            44.550386          44.550386   
2024-01-31                      <NA>       <NA>               <NA>   
2024-02-29                      <NA>       <NA>               <NA>   
2024-03-30                      <NA>       <NA>               <NA>   
2024-03-31                      <NA>       <NA>               <NA>   
2024-04-01                      <NA>       <NA>               <NA>   
2024-04-30                      <NA>       <NA>               <NA>   
2024-05-31                      <NA>       <NA>               <NA>   
2024-06-30                      <NA>   

In [ ]:
concat_inst_esg = pd.DataFrame()
for instrument in esg_scores:
    ins_df = esg_Scores[instrument]
    ins_df["Intrument"] = instrument
    

In [35]:
spx = rd.get_data('0#.SPX',['TR.TRBCEconomicSector','TR.TRESGScore(Period=FY0)', 
                                 'TR.TRESGCScore(Period=FY0)','TR.TRESGCControversiesScore(Period=FY0)', 'TR.HeadquartersRegion'])

spx

,Instrument,TRBC Economic Sector Name,ESG Score,ESG Combined Score,ESG Controversies Score,Region of Headquarters
0,POOL.OQ,Consumer Cyclicals,50.640994,50.640994,100.0,Americas
1,CHRW.OQ,Industrials,41.165895,41.165895,100.0,Americas
2,AJG.N,Financials,50.61887,50.61887,100.0,Americas
3,CNP.N,Utilities,46.144726,32.447363,18.75,Americas
4,AMCR.N,Basic Materials,73.264793,68.299063,63.333333,Europe
...,...,...,...,...,...,...
498,PSX.N,Energy,68.956784,44.663577,20.37037,Americas
499,SCHW.N,Financials,53.71592,48.199424,42.682927,Americas
500,SNPS.OQ,Technology,60.671843,60.671843,81.578947,Americas
501,J.N,Industrials,78.012206,62.119311,46.226415,Americas
